## Creando una vista hardcodeada

Views - Vistas

- Vistas basadas en funciones
- Vistas basadas en clases

### Creando una aplicación

python manage.py startapp <nombre_de_la_app>

python manage.py startapp ecommerce
    
- funciones - nambrarlas con minusculas
- clases - nombrarlas con mayusculas


## ecommerce/views.py



In [ ]:
from django.shortcuts import render
from django.http import HttpResponse


# Create your views here.
def home(request):
    html = """
    <!DOCTYPE html>
    <html>
        <head>
            <style>
                h1 {color: blue}
            </style>
        </head>

        <body>
            <h1>Hola Mundo</h1>
        </body>
    </html>
    """
    return HttpResponse(html)

## ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.home, name="home"),
]

## config/urls.py

In [ ]:
from django.contrib import admin
from django.urls import include
from django.urls import path

urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("ecommerce", include("ecommerce.urls")),
    path("admin/", admin.site.urls),
]

## Respuesta http y redireccionamiento

middlewares - procesos intermedios entre un request y un response

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect


# Create your views here.
def home(request):
    html = """
    <!DOCTYPE html>
    <html>
        <head>
            <style>
                h1 {color: blue}
            </style>
        </head>

        <body>
            <h1>Hola Mundo</h1>
        </body>
    </html>
    """
    return HttpResponse(html)

# def home(request):
#     response = HttpResponse()    
#     response.write("<p>Página No Encontrada</p>")
#     response.status_code = 404

#     return response

def redirect_to_test(request):
    return HttpResponseRedirect("/ecommerce")

## CRUD y Vistas

Forma dinámica de usar las vistas

CRUD - Create, Retrieve, Update, Delete
(Crear, Consultar, Actualizar, Eliminar)

- Crear Modelos
- Agregar la app a INSTALLED_APPS
- Crear y aplicar migraciones
- Agregar admin

### ecommerce/modles.py


In [ ]:
from django.db import models

# Create your models here.

class ProducModel(models.Model):
    title = models.TextField()
    price = models.FloatField()

### config/settings.py

In [ ]:
# Application definitions
INSTALLED_APPS = [
    "ecommerce.apps.EcommerceConfig",
    "pages.apps.PagesConfig",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

## Cómo correr migraciones

- python manage.py makemigrations
- python manage.py migrate

## Cómo registrar modelos en el admin


In [ ]:
from django.contrib import admin
from .models import ProducModel

# Register your models here.
admin.site.register(ProducModel)

## Cómo crear crear un superuser para el admin

- python manage.py createsuperuser

## Tipos básicos de Vistas

- List View - donde ves los usuarios creados
- Create view - donde puedes insertar los datos para crear un usuario
- Retrieve and Update - si le doy click a un usuario creado y lo modifico
- Delete view - botón rojo para eliminar y confirmar

### ecommerce/view.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProducModel

# Create your views here.
def product_model_list_view(request):
    queryset = ProducModel.objects.all()
    template = "ecommerce/list-view.html"
    context = {
        "products" : queryset
    }
    return render(request, template, context)

### templates/ecommerce/list-view.html


In [ ]:
<h1>
Vista de listado
</h1>

{% for product in products %}
    <li>
        {{ product.title }} {{ product.price }}    
    </li>
{% endfor %}

### Protegiendo los endpoints

## Vista de Detalle

#### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProducModel

# Create your views here.

def product_model_detail_view(request, product_id):
    instance = get_object_or_404(ProducModel, id=product_id)
    context = {
        "products" : instance
    }

    template = "ecommerce/detail-view.html"
    
    return render(request, template, context)

#### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
]

#### templates/ecommerce/detail-view.html

In [ ]:
<h1>
    {{ product.title }}
    {{ product.price }}
</h1>

#### templates/ecommerce/list-view.html


In [ ]:
<h1>
Vista de listado
</h1>

{% for product in products %}
    <li>
        <a href="/ecommmerce/{{ product.id }}"> {{ product.title }}</a>    
    </li>
{% endfor %}

## Vista de Ceación

#### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect
from django.contrib import messages

from .models import ProducModel
from .form import ProducModelForm

def product_model_create_view(request):
    form = ProducModelForm(request.POST or None)

    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Producto creado con exito")
        return HttpResponseRedirect("/ecommerce/{product.id}".format(product_id=instance.id))
    
    context = {
        "form":form
    }

    template = "ecommerce/create-view.html"
    return render(request, template, context)

### ecommerce/forms.py


In [ ]:
from django import forms
from .models import ProducModel

class ProducModelForm(forms.ModelForm):
    class Meta:
        model = ProducModel
        field[
            "title",
            "price"
        ]

#### templates/ecommerce/create-view.html

In [ ]:
{% if messages %}
<ul class="messages">
    {% for message in messages %}
    <li{% if message.tags %} class="{{ message.tags }}"{% endif %}>{{ message }}</li>
    {% endfor %}
</ul>
{% endif %}

## Vista de Actualizacón

#### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect
from django.contrib import messages

from .models import ProducModel
from .forms import ProducModelForm

# Create your views here.
def product_model_update_view(request, product_id=None):
    instance = get_object_or_404(ProducModel, id=product_id)
    form = ProducModelForm(request.POST or None, instance=instance)

    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Producto actualizado con exito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
    
    context = {
        "form":form
    }

    template = "ecommerce/update-view.html"
    return render(request, template, context)

#### templates/ecommerce/update-view.html

In [ ]:
<h1>
    Actualización de producto {{ form.instance.title }}
</h1>

{{ form.instance.title }}

<form method="POST" action="">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Actualizar"> 
</form>

#### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create", views.product_model_create_view, name="create"),
    path("<int:product_id>/edit/", views.product_model_update_view, name="update"),
]

### Vista para eliminar

### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect
from django.contrib import messages

from .models import ProducModel
from .forms import ProducModelForm

# Create your views here.
def product_model_delete_view(request, product_id):
    instance = get_object_or_404(ProducModel, id=product_id)
    if request.method == "POST":
        instance.delete()
        HttpResponseRedirect("/ecommerce/")
        messages.succes("Producto eliminado")
    context = {
        "product" : instance
    }

    template = "ecommerce/delete-view.html"

    return render(request, template, context)

#### templates/ecommerce/delete-view.html


In [ ]:
{% include "ecommerce/messages.html" %}

<h1>
    Eliminar {{ product.title }}
</h1>

<form method="POST" action="">
    {% csrf_token %}
    ¿Estás seguro que quieres eliminar el producto: {{ product.title }}??
    <input type="submit" value="Eliminar">
    <a href="/ecommerce/{{ product.id }}">Cancelar 
</form>

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_detail_view, name="detail"),
    path("create", views.product_model_create_view, name="create"),
    path("<int:product_id>/edit/", views.product_model_update_view, name="update"),
    path("<int:product_id>/delete/", views.product_model_delete_view, name="dlete"),
]

### Búsqueda en la Vista de Listado

#### templates/ecommerce/search.html

In [ ]:
<form method="GET" action="/ecommerce/"> 
    <input type="text" name="q" placeholder="Buscar">
</form>

#### templates/ecommerce/list-view.html

In [ ]:
{% include "ecommerce/messages.html" %}

{% include "ecommerce/search.html" %}

<h1>
Vista de listado
</h1>

{% for product in products %}
    <li>
        <a href="/ecommerce/{{ product.id }}"> {{ product.title }}</a>    
    </li>
{% endfor %}

#### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect
from django.contrib import messages
from django.db.models import Q

from .models import ProducModel
from .forms import ProducModelForm

# Create your views here.

def product_model_list_view(request):
    query = request.GET.get("q", None)
    queryset = ProducModel.objects.all()
    if query is not None:
        queryset = queryset.filter(
            Q(title__contains=query) |
            Q(price__contains=query)
            )
    
    template = "ecommerce/list-view.html"
    
    context = {
        "products" : queryset
    }

    if request.user.is_authenticated:
        template = "ecommerce/list-view.html"
    else:
        template = "ecommerce/list-view-public,html"

    return render(request, template, context)

## ACTIVIDAD

#### ecommerce/models.py

In [ ]:
from django.db import models

# Create your models here.

class ProducModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField()
    color = models.TextField()


- python manage.py makemigrations
- python manage.py migrate

In [ ]:
from django.db import migrations, models


class Migration(migrations.Migration):

    dependencies = [
        ("ecommerce", "0001_initial"),
    ]

    operations = [
        migrations.AlterField(
            model_name="producmodel",
            name="color",
            field=models.TextField(verbose_name=""),
        ),
        migrations.AlterField(
            model_name="producmodel",
            name="description",
            field=models.TextField(verbose_name=""),
        ),
    ]


#### ecommerce/forms.py

In [ ]:
from django import forms
from .models import ProducModel

class ProducModelForm(forms.ModelForm):
    class Meta:
        model = ProducModel
        fields = [
            "title",
            "price",
            "description",
            "color",
        ]

#### templates/ecommerce/detail-view.html

In [ ]:
{% include "ecommerce/messages.html" %}

<h1>
    {{ product.title }}
    {{ product.price }}
</h1>

<h2>
    Descripción
</h2>

<p>
    {{ product.description }}
</p>

<h2>
    Color
</h2>

<p>
    {{ product.color }}
</p>